In [1]:
#!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, contains, when, sum, avg, round,max, row_number, desc
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder.appName("ExtractNestedData").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/03 18:48:22 WARN Utils: Your hostname, Benjamins-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.35 instead (on interface en0)
25/06/03 18:48:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/03 18:48:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [117]:
df = spark.read.csv("Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv",header = True)
df.show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|
|  2|Breast Diagnosis|    PR|BreastDx-01-0002|    1|MRI BREAST, BILAT...|36277-2|IMAGING|    MR|2008-11-25|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|
|  4|Breast Diagnosis|    PR|BreastDx-01-0002|    3|PET SKULL-THIGH P...|44139-4|IMAGING|    PT|2008-11-29|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|
|  6|Breast Diagnosis|    PR|BreastDx-01-0002|    5|PET SKULL-THIGH P...|44139-4|IMAGING|    PT|2008-11-29|
|  7|Breast Diagnosis|    PR

25/06/03 19:52:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [9]:
df.count()

1670

In [10]:
df.columns

['_c0',
 'STUDYID',
 'DOMAIN',
 'USUBJID',
 'PRSEQ',
 'PRTRT',
 'PRDECOD',
 'PRCAT',
 'PRSCAT',
 'PRSTDTC']

In [20]:
cat = df.select(col("PRCAT"))
cat.distinct().show()

+-------+
|  PRCAT|
+-------+
|IMAGING|
+-------+



In [21]:
import pandas as pd

In [24]:
df.filter(col("PRSCAT") == "CT").show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|
|  8|Breast Diagnosis|    PR|BreastDx-01-0002|    7|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|
| 18|Breast Diagnosis|    PR|BreastDx-01-0005|    3|PET CT TUMOR, WHO...|46305-9|IMAGING|    CT|2008-08-01|
| 19|Breast Diagnosis|    PR|BreastDx-01-0005|    4|PET CT TUMOR, WHO...|46305-9|IMAGING|    CT|2008-08-01|
| 20|Breast Diagnosis|    PR|BreastDx-01-0005|    5|PET CT TUMOR, WHO...|46305-9|IMAGING|    CT|2008-08-01|
| 26|Breast Diagnosis|    PR

25/06/03 19:02:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [28]:
df[df["PRSCAT"] == "CT"].show() #PD

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|
|  8|Breast Diagnosis|    PR|BreastDx-01-0002|    7|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|
| 18|Breast Diagnosis|    PR|BreastDx-01-0005|    3|PET CT TUMOR, WHO...|46305-9|IMAGING|    CT|2008-08-01|
| 19|Breast Diagnosis|    PR|BreastDx-01-0005|    4|PET CT TUMOR, WHO...|46305-9|IMAGING|    CT|2008-08-01|
| 20|Breast Diagnosis|    PR|BreastDx-01-0005|    5|PET CT TUMOR, WHO...|46305-9|IMAGING|    CT|2008-08-01|
| 26|Breast Diagnosis|    PR

25/06/03 19:03:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [33]:
from pyspark.sql.functions import rank, row_number,dense_rank

In [43]:
dense_rank_df = df.withColumn("rank_over_subject_PRSCAT",dense_rank().over(Window.partitionBy("USUBJID","PRSCAT").orderBy("PRSTDTC")))

In [44]:
dense_rank_df.show()

25/06/03 19:08:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|rank_over_subject_PRSCAT|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                       1|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       1|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       1|
|  8|Breast Diagnosis|    PR|BreastDx-01-0002|    7|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       1|
|  9|Breast Diagnosis|    PR|BreastDx-01-0002|    8|MA SURGICAL SPECI

In [52]:
dense_rank_CT_df = df.withColumn("rank_over_subject_PRSCAT",when(col("PRSCAT") == "CT",
                                                              dense_rank()
                                                              .over(Window.partitionBy("USUBJID","PRSCAT").orderBy("PRSTDTC"))).otherwise(0))

In [53]:
dense_rank_CT_df.show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|rank_over_subject_PRSCAT|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                       0|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       1|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       1|
|  8|Breast Diagnosis|    PR|BreastDx-01-0002|    7|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       1|
|  9|Breast Diagnosis|    PR|BreastDx-01-0002|    8|MA SURGICAL SPECI

25/06/03 19:13:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [54]:
dense_rank_CT_df_Subject = df.withColumn("rank_over_subject_PRSCAT",when(col("PRSCAT") == "CT",
                                                              dense_rank()
                                                              .over(Window.partitionBy("USUBJID").orderBy("PRSTDTC"))).otherwise(0))

In [55]:
dense_rank_CT_df_Subject.show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|rank_over_subject_PRSCAT|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                       0|
|  2|Breast Diagnosis|    PR|BreastDx-01-0002|    1|MRI BREAST, BILAT...|36277-2|IMAGING|    MR|2008-11-25|                       0|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       2|
|  4|Breast Diagnosis|    PR|BreastDx-01-0002|    3|PET SKULL-THIGH P...|44139-4|IMAGING|    PT|2008-11-29|                       0|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P

25/06/03 19:14:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [68]:
rank_CT_df = df.withColumn("rank_over_subject_PRSCAT",when(col("PRSCAT") == "CT",
                                                              rank()
                                                              .over(Window.partitionBy("USUBJID").orderBy("USUBJID","PRSTDTC"))).otherwise(0))

In [69]:
rank_CT_df.orderBy("USUBJID").show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|rank_over_subject_PRSCAT|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                       0|
|  2|Breast Diagnosis|    PR|BreastDx-01-0002|    1|MRI BREAST, BILAT...|36277-2|IMAGING|    MR|2008-11-25|                       0|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       2|
|  4|Breast Diagnosis|    PR|BreastDx-01-0002|    3|PET SKULL-THIGH P...|44139-4|IMAGING|    PT|2008-11-29|                       0|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P

25/06/03 19:18:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [85]:
rank_CT_df = df.withColumn("rank_over_subject_PRSCAT",rank().over(Window.partitionBy("USUBJID").orderBy("PRSTDTC")))

In [86]:
rank_CT_df.show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|rank_over_subject_PRSCAT|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                       1|
|  2|Breast Diagnosis|    PR|BreastDx-01-0002|    1|MRI BREAST, BILAT...|36277-2|IMAGING|    MR|2008-11-25|                       1|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       2|
|  4|Breast Diagnosis|    PR|BreastDx-01-0002|    3|PET SKULL-THIGH P...|44139-4|IMAGING|    PT|2008-11-29|                       2|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P

25/06/03 19:23:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [87]:
rank_CT_df = df.withColumn("rank_over_subject_PRSCAT",dense_rank().over(Window.partitionBy("USUBJID").orderBy("PRSTDTC")))

In [88]:
rank_CT_df.show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|rank_over_subject_PRSCAT|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                       1|
|  2|Breast Diagnosis|    PR|BreastDx-01-0002|    1|MRI BREAST, BILAT...|36277-2|IMAGING|    MR|2008-11-25|                       1|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       2|
|  4|Breast Diagnosis|    PR|BreastDx-01-0002|    3|PET SKULL-THIGH P...|44139-4|IMAGING|    PT|2008-11-29|                       2|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P

25/06/03 19:24:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [91]:
dense_rank_CT_df_Subject = df.withColumn("rank_over_subject_PRSCAT",when(col("PRSCAT") == "CT",
                                                              row_number()
                                                              .over(Window.partitionBy("USUBJID").orderBy("USUBJID"))).otherwise(0))

In [90]:
dense_rank_CT_df_Subject.show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|rank_over_subject_PRSCAT|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                       0|
|  2|Breast Diagnosis|    PR|BreastDx-01-0002|    1|MRI BREAST, BILAT...|36277-2|IMAGING|    MR|2008-11-25|                       0|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       2|
|  4|Breast Diagnosis|    PR|BreastDx-01-0002|    3|PET SKULL-THIGH P...|44139-4|IMAGING|    PT|2008-11-29|                       0|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P

25/06/03 19:25:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [94]:
dense_rank_CT_df_Subject = df.withColumn("rank_over_subject_PRSCAT",when(col("PRSCAT") == "CT",
                                                              row_number()
                                                              .over(Window.partitionBy("USUBJID","PRSCAT").orderBy("USUBJID"))).otherwise(0))

In [102]:
dense_rank_CT_df_Subject.show(30)

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|rank_over_subject_PRSCAT|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                       0|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       1|
|  5|Breast Diagnosis|    PR|BreastDx-01-0002|    4|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       2|
|  8|Breast Diagnosis|    PR|BreastDx-01-0002|    7|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                       3|
|  9|Breast Diagnosis|    PR|BreastDx-01-0002|    8|MA SURGICAL SPECI

25/06/03 19:50:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [103]:
scat = df.select(col("PRTRT"))
scat.distinct().show(40,truncate=False)

+----------------------------------------+
|PRTRT                                   |
+----------------------------------------+
|MR BRAIN WITHOUT AND WITH CONTRAST D159 |
|MR BRAIN WITHOUT AND WITH CONTRAST D799 |
|MR BRAIN WITHOUT AND WITH CONTRAST D98  |
|MR BRAIN WITHOUT AND WITH CONTRAST D30  |
|MR BRAIN WITHOUT AND WITH CONTRAST D212 |
|MR BRAIN WITHOUT AND WITH CONTRAST D702 |
|MR BRAIN WITHOUT AND WITH CONTRAST D355 |
|MR BRAIN WITHOUT AND WITH CONTRAST D222 |
|MR BRAIN WITHOUT AND WITH CONTRAST D270 |
|MR BRAIN WITHOUT AND WITH CONTRAST D182 |
|MR BRAIN WITHOUT AND WITH CONTRAST D294 |
|MR BRAIN WITHOUT AND WITH CONTRAST D591 |
|MRI PRE&POST INF BREASTMR               |
|OUTSIDE MR - HEAD D97                   |
|MRBRUR                                  |
|MR BRAIN WITHOUT AND WITH CONTRAST D647 |
|3607 ACRIN 6657 LT BREAST/GAD           |
|Breast Bil tumor ax                     |
|OUTSIDE MR - HEAD D94                   |
|MR BRAIN WITHOUT AND WITH CONTRAST D1039|
|MR BRAIN W

In [123]:
win = Window.partitionBy("USUBJID").orderBy('PRSTDTC')

DSC = df.withColumn(
    'Digital Spot Compression: 9286464',
    when((col('PRDECOD') == '46342-2') ,lit(1)).otherwise(0))

In [124]:
DSC.show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+---------------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|Digital Spot Compression: 9286464|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+---------------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                                0|
|  2|Breast Diagnosis|    PR|BreastDx-01-0002|    1|MRI BREAST, BILAT...|36277-2|IMAGING|    MR|2008-11-25|                                0|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                                0|
|  4|Breast Diagnosis|    PR|BreastDx-01-0002|    3|PET SKULL-THIGH P...|44139-4|IMAGING|    PT|2008-11-29|                                0|
|  5|B

25/06/03 19:55:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv


In [134]:
win = Window.partitionBy("USUBJID").orderBy("PRSTDTC")

DSC_2 = df.withColumn(
    'Digital Spot Compression: 9286464',
    when(
        max(when((col('PRDECOD') == '46342-2') ,lit(1)).otherwise(0))
        .over(win) == 1, '46342-2 present').otherwise("Not present")
)

In [133]:
DSC_2.show()

+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+---------------------------------+
|_c0|         STUDYID|DOMAIN|         USUBJID|PRSEQ|               PRTRT|PRDECOD|  PRCAT|PRSCAT|   PRSTDTC|Digital Spot Compression: 9286464|
+---+----------------+------+----------------+-----+--------------------+-------+-------+------+----------+---------------------------------+
|  1|Breast Diagnosis|    PR|BreastDx-01-0001|    1|MRI Breast Bilate...|36277-2|IMAGING|    MR|2008-08-12|                      Not present|
|  2|Breast Diagnosis|    PR|BreastDx-01-0002|    1|MRI BREAST, BILAT...|36277-2|IMAGING|    MR|2008-11-25|                  46342-2 present|
|  3|Breast Diagnosis|    PR|BreastDx-01-0002|    2|PET SKULL-THIGH P...|46305-9|IMAGING|    CT|2008-11-29|                  46342-2 present|
|  4|Breast Diagnosis|    PR|BreastDx-01-0002|    3|PET SKULL-THIGH P...|44139-4|IMAGING|    PT|2008-11-29|                  46342-2 present|
|  5|B

25/06/03 19:58:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
 Schema: _c0, STUDYID, DOMAIN, USUBJID, PRSEQ, PRTRT, PRDECOD, PRCAT, PRSCAT, PRSTDTC
Expected: _c0 but found: 
CSV file: file:///Users/benjaminbrooke/Downloads/dicubed_sdtm_csv_2019-05-30/pr.csv
